# Pycrash

In [ ]:
import os
path_parent = os.path.dirname(os.getcwd())
data_directory = os.path.join(path_parent, "data")
os.chdir(path_parent)

# %% Import Modules
import matplotlib.pyplot as plt
from matplotlib.pyplot import text
from IPython.display import display
from src.functions import EnergyDV, SpringSeriesKeff, cipriani
from src.sdof_model import SDOF_Model
from src.project import Project, project_info, load_project
from src.vehicle import Vehicle
from scipy import signal
from scipy import integrate
import pandas as pd
import numpy as np
import pickle
import json

pd.options.display.max_columns = None
from IPython import get_ipython
#get_ipython().run_line_magic('matplotlib', 'qt')

### Instanstiate Project
Prompts user for inputs  
`name` - used to save / load project  
`description` - free text to describe type of accident eg.: front to rear, sideswipe etc.  
`simulation type` - single or multi vehicle  
`impact type` - if multi vehicle an impact type type is chosen: "SS", "IMPC", "SDOF"  
`note` - free text note for future reference  

In [ ]:
proj = Project()

### Instantiate Vehicles
There are multiple methods to create a `Vehicle` object  
vehicle = Vehicle('name', input_dict = Null) 
1. `input_dict` see below - predefined python dictionary for inputs, only those necessary for future analysis
needs to be supplied
2. After creating vehicle object "veh1" use `veh1.manual_specs()` for user prompts of all inputs
3. After creating vehicle object "veh1" use `veh1.load_specs('filename.csv') to load specs from csv file with correct format

In [ ]:
# Vehicle 1

vehicle1_input_dict = {"year":2011,
"make":"International",
"model":"DuraStar 4400",
"weight":80000,
"vin":"1HSMTAAN1BH324837",
"brake":0,
"steer_ratio":0,
"init_x_pos":0,
"init_y_pos":0,
"head_angle":0,
"v_width":0,
"v_length":0,
"hcg":0,
"lcgf":0,
"lcgr":0,
"wb":0,
"track":0,
"f_hang":0,
"r_hang":0,
"tire_d":0,
"tire_w":0,
"izz":0,
"fwd":0,
"rwd":1,
"awd":0,
"A":0,
"B":0,
"k":7143,
"L":0,
"c":0,
"vx_initial":1,
"vy_initial":0}

veh1 = Vehicle('International', input_dict = vehicle1_input_dict)

# Vehicle 2

vehicle2_input_dict = {"year":1998,
"make":"Ford",
"model":"Explorer",
"weight":3309,
"vin":"1FMYU22X2WUC31945",
"brake":0,
"steer_ratio":0,
"init_x_pos":0,
"init_y_pos":0,
"head_angle":0,
"v_width":0,
"v_length":0,
"hcg":0,
"lcgf":0,
"lcgr":0,
"wb":0,
"track":0,
"f_hang":0,
"r_hang":0,
"tire_d":0,
"tire_w":0,
"izz":0,
"fwd":0,
"rwd":1,
"awd":0,
"A":0,
"B":0,
"k":7143,
"L":0,
"c":0,
"vx_initial":0,
"vy_initial":0}

veh2 = Vehicle('Ford', input_dict = vehicle2_input_dict)

#### Get python dictionary of vehicle inputs with `veh1.input_dict()`

In [ ]:
veh1.input_dict()

### Single Degree of Freedom Model (SDOF)
- simulate vehicle to vehicle interaction with mutual stiffness
- braking effects

In [ ]:
model_inputs = {'name':'run1',
               'cor':0.2,
               'k':1000,
               'tstop':0.1}

model_run = sdof_model(veh1, veh2, model_inputs=None)

creates a static copy of each vehicle object

### Change functions to take list of simulations

In [ ]:
# %% Stiffness Values for 3 models
veh1_k = [7143, 7143, 7143]
veh2_k = [3230, 5220, 7143]
k_mutual = []
for i in range(len(veh1_k)):
    k_mutual.append(SpringSeriesKeff(veh1_k[i], veh2_k[i]))

# %% create model inputs
v1_vx_initial = [1.5, 2, 2.5]      # initial speeds for striking vehicle
colorList = ['k', 'g', 'b']
run_list_names = ['run1', 'run2', 'run3']
cor_list = [0.2 , 0.2, 0.2]  # low restitution from sideswipe

# %% Run three models and plot force-deflection

fig = plt.figure(figsize = (14,12))
plt.title('Mutual Force', fontsize=20)
models =[None] * len(run_list_names)  # create empty list for model runs

for i in range(len(v1_vx_initial)):

    model_inputs = {"name":run_list_names[i],
            "k":k_mutual[i],
            "cor":cor_list[i], 
            "tstop":0.12
        }
    veh1.vx_initial = v1_vx_initial[i]
    veh1.k = veh1_k[i]
    veh2.k = veh2_k[i]
    models[i] = SDOF_Model(veh1, veh2, model_inputs)

    # Calculate vehicle specific crush
    models[i].model_result['veh1_dx'] = models[i].model_result.dx * (models[i].k / veh1.k)
    models[i].model_result['veh2_dx'] = models[i].model_result.dx * (models[i].k / veh2.k)
    print(f'Peak Explorer Acceleration Model {models[i].name} = {models[i].model_result.a2.max() / 32.2:.2f} g')

    plt.plot(models[i].model_result.t, -1 * models[i].model_result.dx * 12, label = f'F350 F-dx Vi={v1_vx_initial[i]} mph', color = colorList[i])
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    plt.xlabel('Time (s)', fontsize=20)
    plt.ylabel('Mutal Crush (in)', fontsize=20)
    plt.grid(which='both', axis='both')
    plt.legend(fontsize=14)

    plt.hlines(0.9, 0, 0.16, linestyles='--', colors='r')
    plt.text(0.06, 0.9 + 0.02, 'Subject Explorer Deformation', fontdict={'size':16})   
    plt.show()


# %% assign final models
run1, run2, run3 = models


In [ ]:
# %% comparison velocity plot 
veh1_colors = ['g', 'g', 'g']
veh2_colors = ['b', 'b', 'b']

def velocity_compare(run1, run2, run3, veh1_colors, veh2_colors, fill_diff = False, show_legend = False):
    """
    three model runs will be plotted together
    runs should be arranged low / mid / high
    """
    
    fig = plt.figure(figsize = (14,12))
    plt.title('Vehicle Velocity', fontsize=20)
    plt.plot(run1.model_result.t, run1.model_result.v1 * 0.681818, label = f'{run1.name} - {run1.veh1.name}', color = veh1_colors[0])
    plt.plot(run2.model_result.t, run2.model_result.v1 * 0.681818, label = f'{run1.name} - {run1.veh1.name}', color = veh1_colors[1])
    plt.plot(run3.model_result.t, run3.model_result.v1 * 0.681818, label = f'{run1.name} - {run1.veh1.name}', color = veh1_colors[2])

    plt.plot(run1.model_result.t, run1.model_result.v2 * 0.681818, label = f'{run1.name} - {run1.veh2.name}', color = veh2_colors[0])
    plt.plot(run2.model_result.t, run2.model_result.v2 * 0.681818, label = f'{run1.name} - {run1.veh2.name}', color = veh2_colors[1])
    plt.plot(run3.model_result.t, run3.model_result.v2 * 0.681818, label = f'{run1.name} - {run1.veh2.name}', color = veh2_colors[2])

    if fill_diff:
        plt.fill_between(run1.model_result.t, run1.model_result.v1 * 0.681818, run3.model_result.v1 * 0.681818, alpha=0.3, facecolor=veh1_colors[1])
        plt.fill_between(run1.model_result.t, run1.model_result.v2 * 0.681818, run3.model_result.v2 * 0.681818, alpha=0.3, facecolor=veh2_colors[1])


    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    plt.xlim([0, max(max(run1.model_result.t), max(run2.model_result.t), max(run3.model_result.t))])
    plt.ylim([0, 1 + round(max(max(run1.model_result.v1) * 0.681818, max(run2.model_result.v1) * 0.681818, max(run3.model_result.v1) * 0.681818,
                       max(run1.model_result.v2) * 0.681818, max(run2.model_result.v2) * 0.681818, max(run3.model_result.v2) * 0.681818))])

    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    plt.xlabel('Time (s)', fontsize=20)
    plt.ylabel('Velocity (mph)', fontsize=20)

    if show_legend:
        plt.legend(fontsize=14, frameon = False)

    plt.show()

In [ ]:
# %% Acceleration Plot

def accel_compare(run1, run2, run3, veh1_colors, veh2_colors, fill_diff = False, show_legend = False):
    """
    three model runs will be plotted together
    runs should be arranged low / mid / high
    """

    fig = plt.figure(figsize = (14,12))
    plt.title('Vehicle Acceleration', fontsize=20)
    plt.plot(run1.model_result.t, run1.model_result.a1 / 32.2, label = f'{run1.name} - {run1.veh1.name}', color = veh1_colors[0])
    plt.plot(run2.model_result.t, run2.model_result.a1 / 32.2, label = f'{run1.name} - {run1.veh1.name}', color = veh1_colors[1])
    plt.plot(run3.model_result.t, run3.model_result.a1 / 32.2, label = f'{run1.name} - {run1.veh1.name}', color = veh1_colors[2])

    plt.plot(run1.model_result.t, run1.model_result.a2 / 32.2, label = f'{run1.name} - {run1.veh2.name}', color = veh2_colors[0])
    plt.plot(run2.model_result.t, run2.model_result.a2 / 32.2, label = f'{run1.name} - {run1.veh2.name}', color = veh2_colors[1])
    plt.plot(run3.model_result.t, run3.model_result.a2 / 32.2, label = f'{run1.name} - {run1.veh2.name}', color = veh2_colors[2])

    if fill_diff:
        plt.fill_between(run1.model_result.t, run1.model_result.a1 / 32.2, run3.model_result.v1 / 32.2, alpha=0.3, facecolor=veh1_colors[1])
        plt.fill_between(run1.model_result.t, run1.model_result.a2 / 32.2, run3.model_result.v2 / 32.2, alpha=0.3, facecolor=veh2_colors[1])


    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    plt.xlim([0, max(max(run1.model_result.t), max(run2.model_result.t), max(run3.model_result.t))])
    plt.ylim([-1 + round(min(min(run1.model_result.a1) / 32.2, min(run2.model_result.a1) / 32.2, min(run3.model_result.a1) / 32.2,
                       min(run1.model_result.a2) / 32.2, min(run2.model_result.a2) / 32.2, min(run3.model_result.a2) / 32.2)), 
              1 + round(max(max(run1.model_result.a1) / 32.2, max(run2.model_result.a1) / 32.2, max(run3.model_result.a1) / 32.2,
                       max(run1.model_result.a2) / 32.2, max(run2.model_result.a2) / 32.2, max(run3.model_result.a2) / 32.2))])

    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    plt.xlabel('Time (s)', fontsize=20)
    plt.ylabel('Acceleration (g)', fontsize=20)

    if show_legend:
        plt.legend(fontsize=14, frameon = False)

    plt.show()

In [ ]:
# %% Plot velocities
velocity_compare(run1, run2, run3, veh1_colors, veh2_colors, fill_diff = True)
accel_compare(run1, run2, run3, veh1_colors, veh2_colors, fill_diff = False)

In [ ]:
# %% Force-Deflection
run_colors = ['b', 'g', 'k']
fig = plt.figure(figsize = (14,12))
plt.title('Mutual Force - Deflection', fontsize=20)

plt.plot(run1.model_result.dx * -12, run1.model_result.springF, label = f'{run1.name} - {run1.veh1.name}', color = run_colors[0])
plt.plot(run2.model_result.dx * -12, run2.model_result.springF, label = f'{run2.name} - {run1.veh1.name}', color = run_colors[1])
plt.plot(run3.model_result.dx * -12, run3.model_result.springF, label = f'{run3.name} - {run1.veh1.name}', color = run_colors[2])

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlim([0, -12 * min(min(run1.model_result.dx), min(run2.model_result.dx), min(run3.model_result.dx))])
plt.ylim([0, round(max(max(run1.model_result.springF), max(run2.model_result.springF), max(run3.model_result.springF)))])

ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)


plt.xlabel('Mutal Crush (in)', fontsize=20)
plt.ylabel('Force (lb)', fontsize=20)
#plt.legend(fontsize=14)

plt.vlines(0.9, 0, 7000, linestyles='--', colors='r')
plt.text(0.85, 4000, 'Subject Explorer Deformation', fontdict={'size':16} , rotation=90)   
plt.show()

### Sideswipe Model
- Normal / tangential force calculation for sideswipe interaction
- Steering / braking effects

### Impulse Momentum Collision
- Two vehicle motion simulation with impact detection
- impulse-momentum collision response

### Single Vehicle Motion
- calculate vehicle motion given initial conditions and driver inputs
- driver inputs are assigned to vehicle using two methods:
    - predefined time, steer and brake arrays - should be dataframe?

In [ ]:
# calculate vehicle motion for Vehicle 1
v1_in = premotion(1)
plot_inputs(v1_in)

In [ ]:
# calculate vehicle motion for Vehicle 2
v2_in = premotion(2)
plot_inputs(v2_in)

#### Vehicle Pre-impact Motion

In [ ]:
veh_1_pre_motion = vehicle_model(1)
plot_vehicle_pre_motion(v1_in, veh_1_pre_motion)

In [ ]:
draw_vx, draw_vy, draw_gx, draw_gy = global_frame_df(veh_1_pre_motion, 1)
draw_vehicle_motion(draw_gx, draw_gy, 1, len(draw_gx)-1)

In [ ]:
draw_vx, draw_vy, draw_gx, draw_gy = global_frame_df(veh_1_pre_motion, 1)
fig0 = draw_vehicle_motion(draw_gx, draw_gy, 1, len(draw_gx)-1)
fig1 = draw_vehicle_motion(draw_gx, draw_gy, 1, 0)

In [ ]:
draw_vx, draw_vy, draw_gx, draw_gy = global_frame_df(veh_1_pre_motion, 1)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize = (15,10))
plotRange = [20, 40]

custom_xlim = (0, 350)
custom_ylim = (10, -160)
plt.setp(axes, xlim=custom_xlim, ylim=custom_ylim)

for ax, i in zip(axes.flatten(), plotRange):
    draw_vehicle_motion_loop(draw_gx, draw_gy, 1, ax, i)

In [ ]:
fig = plt.figure(figsize=(15,10))
plt.xlim([0, 550])
plt.ylim([-350, 0])
plt.gca().invert_yaxis()
fig = draw_vehicle_motion(draw_gx, draw_gy, 1, 0)

In [ ]:
#%% Plot vehicle in vehicle frame
from src.plots import draw_vehicle
draw_vehicle(draw_vx, draw_vy, 42)

In [ ]:
fig.savefig('D:\\OneDrive\\Vehicle_Dynamics\\plots\\global.png')

### Import Scene Image

In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
# load DX, DY
v1_pre = veh_1_pre_motion
theta_shift = -10 * math.pi / 180
x_shift = 200
y_shift = 240
# rotate and translate
for i in (range(len(v1_pre))):
    v1_pre.loc[i, 'Dx_'] = x_shift + v1_pre.loc[i, 'Dx']*math.cos(theta_shift) - v1_pre.loc[i, 'Dy']*math.sin(theta_shift)
    v1_pre.loc[i, 'Dy_'] = y_shift + v1_pre.loc[i, 'Dx']*math.sin(theta_shift) + v1_pre.loc[i, 'Dy']*math.cos(theta_shift)

In [ ]:
img = plt.imread('pre_imapct_motion.jpg')
#plt.scatter(x,y,zorder=1)
# height = 333.17 ft, 
# width = 544.51 ft

fig = plt.figure(figsize=(19,15))
plt.scatter(v1_pre.Dx_, v1_pre.Dy_, s=5)
plt.imshow(img, zorder=0, extent=[0, 544.41, 333.17, 0])
plt.show()